In [1]:

import pandas as pd
import os

def filtrar_df_lista(df, str_columna, lista):
    
    '''
    INPUTS: 
    
    df : DataFrame a filtrar
    str_columna : columna por la que se quiere filtrar
    lista : lista de elementos que se quieren ver del df
    
    OUTPUT:
    
    df filtrado y reindexado
    
    '''
    
    import pandas as pd
    import os
    
    new_df = pd.DataFrame()

    new_df = new_df.append([df[df[str_columna] == e] for e in lista])

    return new_df.reset_index(drop = True)
    
    
 
 
def adjust_columns_width(sheet):
    
    """
    Para una hoja de excel dada, ajusta el ancho de las columnas según la longitud de cada una de ellas
    
    """
    
    for column_cells in sheet.columns:
        new_column_length = max(len(str(cell.value)) for cell in column_cells)
        new_column_letter = (get_column_letter(column_cells[0].column))
        if new_column_length > 0:
            sheet.column_dimensions[new_column_letter].width = new_column_length*1.23
            
    sheet.sheet_view.zoomScale = 60
    
    
def diferenciar_columnas_iguales(columns):

    '''
    Si se tiene un df con columnas del mismo nombre, devolverá las mismas columnas pero añadiendo '_n' siendo n un contador.
    Por ejemplo: [Codigo, valor, valor, valor, valor_final, valor_final] --> [Codigo, valor_0, valor_1, valor_2, valor_final_0, valor_final_1]
    
    INPUT: Lista con las columnas del DataFrame
    OUTPUT: Lista con las columnas diferenciadas con un número al final 
    
    '''
    import collections
    
    cols_repetidas = [k for k, v in dict(collections.Counter(columns)).items() if v > 1]

    for c_r in cols_repetidas:
        cont = 0
        for c in range(len(columns)):
            if columns[c] == c_r:
                columns[c] = columns[c] + '_' + str(cont)
                cont += 1

    return columns


def recortar_agrupados(df, column, name_output_directory):
    
        
    '''
    Para un DataFrame dado y una columna seleccionada, vuelca en una carpeta en .xlsx el mismo dataframe recortado
    para cada valor único de cada columna dada.
    Es decir, como si desde excel, filtrásemos uno por uno por cada valor de la columna y guardásemos un archivo por cada valor.
    
    INPUTS: 
    
    df : DataFrame a recortar
    column : columna por la que se quiere recortar
    name_output_directory : nombre de la carpeta en la que se quieren volcar los archivos
    
    OUTPUT:
    
    Excels recortados
    
    '''
    import pandas as pd
    import os 
    
    try:
        os.mkdir(str(name_output_directory))
    except:
        1
    
    df[column] = df[column].fillna('EMPTY')
    
    for value in [e.replace(' ', '_') for e in df[column].unique()]:

        locals()[value] = df[(df[column] == value.replace('_', ' '))]
        
        try:     
            locals()[value].to_excel(name_output_directory + '/' + str(value) + '.xlsx', index = False)
        except:  
            continue
            